In [1]:
from IPython import get_ipython
get_ipython().magic('clear')
import time
import datetime
from re import sub
import numpy as np
import pandas as pd
from decimal import Decimal
import numpy_financial as npf
from bs4 import BeautifulSoup
from selenium import webdriver
from fake_useragent import UserAgent
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
term = 30 # years
rate = 0.0475 # FHA: 4.75%, Conv: 5% as of
pmi_pct = 0.0085 # 0.85% of loan amount
insurance_pct = 0.004 # 0.4% of list price
downpayment_pct = 0.035 # FHA: 3.5%, Conventional: 5% (owner occupied)
closing_costs_pct = 0.01 # 1% is typical for 
expenses = 536.96 # monthly (utilities, reserves, maintenance, etc)

groveSt_rent = 3200 # current home's rental potential
groveSt_piti = 2598.28 # current mortgage pmt

In [3]:
# get estimate rent from Zillow
def getZestimate(driver, address):
    address.replace(" ", "-").replace(",","")
    rent_url = "https://www.zillow.com/rental-manager/price-my-rental/results/" + address
    driver.get(rent_url)
    rent_data = driver.page_source
    rent_soup = BeautifulSoup(rent_data,'html.parser')
    
    try:
        zestimate = rent_soup.find("h2", class_="Text-c11n-8-64-1__sc-aiai24-0 eFOYNq").text
        zestimate = int(Decimal(sub(r'[^\d.]', '', zestimate)))
    except:
        zestimate = 0

    return zestimate



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [/Users/okara/.wdm/drivers/chromedriver/mac64/101.0.4951.41/chromedriver] found in cache
/var/folders/sd/1vc_q83x5rn9jjrd0x47_cc00000gn/T/ipykernel_44459/1884681819.py:35: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(),options=chrome_options)


# Redfin Download csv 

In [ ]:
# get timestamp
ct = str(datetime.datetime.now().strftime("%m-%d-%Y_%H%M%S"))

# use random user agents to try and bypass walls such as CAPTCHA
userAgent = UserAgent().random

chrome_options = webdriver.ChromeOptions()

chrome_options.headless = False
# chrome_options.add_argument('--window-size=1920,1080')
# chrome_options.add_argument(f'user-agent={userAgent}')
chrome_options.add_argument("--proxy-server='direct://'")
chrome_options.add_argument("--proxy-bypass-list=*")
# chrome_options.add_argument("blink-settings=imagesEnabled=false")
chrome_options.add_argument("user-data-dir=C:\\Users\\jabie\\AppData\\Local\\Google\\Chrome\\User Data")
chrome_options.add_experimental_option("prefs", {"download.default_directory" : "C:\\Users\\jabie\\Documents\\Finance\\Real Estate Investing\\code\\input"})


driver = webdriver.Chrome(ChromeDriverManager().install(),options=chrome_options)
# areas: middlesex, suffolk, and norfolk counties
# price range: 0-$1M, beds: 4+, baths: 2+, type: multifamily
# status: "active", "coming soon", no HOA fees, no new construction
# exclude 55+ communities, short sales, and land leases
# sort by newest listings first
# remove 'viewport'
search_url = "https://www.redfin.com/county/1892/NJ/Bergen-County/filter/sort=lo-days,min-price=300k,min-beds=4,min-baths=2"

driver.get(search_url)

time.sleep(3)

search_data = driver.page_source
search_soup = BeautifulSoup(search_data,'html.parser')

download_url = "https://www.redfin.com" + str(search_soup.find("a", id="download-and-save")["href"])

# download the RedFin csv file
driver.get(download_url)



In [4]:
# show the "select file" dialog box

filepath = "data/redfin_2022-05-05-14-41-48_bergen.csv"

# read in RedFin URLs
# df_in = pd.read_csv("test.csv", usecols=[20])
df_in = pd.read_csv(filepath, usecols=[20])

# # read in average region rents by towns/cities
# avg_mass_rent = pd.read_csv("mass_avg_rent2.csv")
# cities = list(avg_mass_rent.iloc[:,0].values)

# set headers for output files
params = {
    'Term': [term],
    'Interest Rate (%)': [rate],
    'PMI (%)': [pmi_pct],
    'Insurance (%)': [insurance_pct],
    'Down Payment (%)': [downpayment_pct],
    'Closing Costs (%)': [closing_costs_pct],
    'Monthly Expenses': [expenses],
    '30GroveSt Mortgage': [groveSt_piti],
    '30GroveSt Rent': [groveSt_rent]
}  

data_out = {
    'MLS': [],
    'Address': [],
    'Town/City': [],
    'Property Type':[],
    'List Price': [],
    'RedFin Estimate': [],
    'Beds': [],
    'Baths':[],
    'SQFT': [],
    'Unit Info':[],
    'RedFin Rent Estimate per Unit':[],
    'MA Average Rent per Unit': [],
    'Zillow Rent Estimate':[],
    'Rent Estimate Used': [],
    'MFH Mortgage Payment': [],
    'Monthly Savings': [],
    'MFH Cash Flow': [],
    'MFH Cash-on-Cash Return (%)': [],
    'Combined Cash Flow': [],
    'Cap Rate (%)': [],
    'Interested?': [],
    'URL': [],
    'Notes':[]
}  

df_out = pd.DataFrame(data_out)
df_param = pd.DataFrame(params)

fn_out = "output_2" + ct + ".csv"
fn_param = "parameters_2" + ct + ".csv"

df_out.to_csv(fn_out, mode='a', index=False, header=True)
df_param.to_csv(fn_param, mode='a', index=False, header=True)

notes = []
rf_rents = []
avg_rents = []
unit_info = []

1 of 350

https://www.redfin.com/NJ/Ridgewood/200-Claremont-Rd-07450/home/35841203
City: Ridgewood is not in average region rent list
2 of 350

https://www.redfin.com/NJ/Montvale/16-Stuyvesant-Rd-07645/home/35796005
City: Montvale is not in average region rent list
3 of 350

https://www.redfin.com/NJ/Ho-Ho-Kus/65-Jacquelin-Ave-07423/home/35767781
City: Ho-ho-kus is not in average region rent list
4 of 350

https://www.redfin.com/NJ/Bergenfield/134-Wilbur-Rd-07621/home/35682017
City: Bergenfield is not in average region rent list
5 of 350

https://www.redfin.com/NJ/Hackensack/136-Kent-St-07601/home/108170679
City: Hackensack is not in average region rent list
6 of 350

https://www.redfin.com/NJ/Westwood/67-Harding-Ave-07675/home/35884694
City: Westwood is not in average region rent list
7 of 350

https://www.redfin.com/NJ/River-Vale/712-Orangeburgh-Rd-07675/home/35847743
City: RiverVale is not in average region rent list
8 of 350

https://www.redfin.com/NJ/Ramsey/238-S-Central-Ave-07446

City: Closter is not in average region rent list
63 of 350

https://www.redfin.com/NJ/Alpine/19-Allison-Rd-07620/home/35675732
City: Alpine is not in average region rent list
64 of 350

https://www.redfin.com/NJ/Hillsdale/7-Royal-Park-Ter-07642/home/35764317
City: Hillsdale is not in average region rent list
65 of 350

https://www.redfin.com/NJ/Demarest/34-Woodland-Rd-07627/home/35698068
City: Demarest is not in average region rent list
66 of 350

https://www.redfin.com/NJ/Wallington/33-Pulaski-Ave-07057/home/35880302
City: Wallington is not in average region rent list
67 of 350

https://www.redfin.com/NJ/Fairview/18-Bergen-Blvd-07022/home/35731845
City: Fairview is not in average region rent list
68 of 350

https://www.redfin.com/NJ/Tenafly/20-Stony-Brook-Rd-07670/home/35875263
City: Tenafly is not in average region rent list
69 of 350

https://www.redfin.com/NJ/Garfield/206-Cambridge-Ave-07026/home/35902046
City: Garfield is not in average region rent list
70 of 350

https://www.redf

City: OldTappan is not in average region rent list
125 of 350

https://www.redfin.com/NJ/Rutherford/9-Courrier-Pl-07070/home/35853743
City: Rutherford is not in average region rent list
126 of 350

https://www.redfin.com/NJ/Lyndhurst/22-Bogle-Dr-07071/home/35904603
City: Lyndhurst is not in average region rent list
127 of 350

https://www.redfin.com/NJ/Old-Tappan/3-Tudor-Rd-07675/home/35812826
City: OldTappan is not in average region rent list
128 of 350

https://www.redfin.com/NJ/Alpine/67-Robin-Ln-07620/home/35675366
City: Alpine is not in average region rent list
129 of 350

https://www.redfin.com/NJ/Mahwah/17-Osborne-Ct-07430/home/35785528
City: Mahwah is not in average region rent list
130 of 350

https://www.redfin.com/NJ/Upper-Saddle-River/11-Anona-Dr-07458/home/35875966
City: UpperSaddleRiver is not in average region rent list
131 of 350

https://www.redfin.com/NJ/ALPINE/10-BUCKINGHAM-DR-07620/home/35675984
City: Alpine is not in average region rent list
132 of 350

https://www

City: OldTappan is not in average region rent list
187 of 350

https://www.redfin.com/NJ/Fairview/93-Fulton-Ave-07022/home/35732004
City: Fairview is not in average region rent list
188 of 350

https://www.redfin.com/NJ/Oradell/403-Grant-Ave-07649/home/35815345
City: Oradell is not in average region rent list
189 of 350

https://www.redfin.com/NJ/Dumont/9-Fern-Ave-07628/home/35702650
City: Dumont is not in average region rent list
190 of 350

https://www.redfin.com/NJ/Dumont/110-Oneida-Ave-07628/home/54477868
City: Dumont is not in average region rent list
191 of 350

https://www.redfin.com/NJ/Mahwah/9-Island-Rd-07430/home/35781828
City: Mahwah is not in average region rent list
192 of 350

https://www.redfin.com/NJ/Ho-Ho-Kus/21-Deerhill-Dr-07423/home/35768775
City: Ho-ho-kus is not in average region rent list
193 of 350

https://www.redfin.com/NJ/Saddle-Brook/161-Nedellec-Dr-07663/home/35859233
City: SaddleBrook is not in average region rent list
194 of 350

https://www.redfin.com/NJ/

WebDriverException: Message: unknown error: cannot determine loading status
from target frame detached
  (Session info: chrome=101.0.4951.54)
Stacktrace:
0   chromedriver                        0x00000001096bf2c9 chromedriver + 5120713
1   chromedriver                        0x000000010964de33 chromedriver + 4656691
2   chromedriver                        0x000000010923d00f chromedriver + 393231
3   chromedriver                        0x0000000109227dec chromedriver + 306668
4   chromedriver                        0x0000000109226efa chromedriver + 302842
5   chromedriver                        0x00000001092273fc chromedriver + 304124
6   chromedriver                        0x0000000109227354 chromedriver + 303956
7   chromedriver                        0x000000010922de92 chromedriver + 331410
8   chromedriver                        0x000000010922e281 chromedriver + 332417
9   chromedriver                        0x00000001092286f6 chromedriver + 308982
10  chromedriver                  

In [ ]:
for i in range(0, len(df_in)):
    
    print(str(i+1) + " of " + str(len(df_in)) + "\n")
        
    url = df_in.iloc[i][0]
    
    print(url)
    
    driver.get(url)
    data = driver.page_source
    soup = BeautifulSoup(data,'html.parser')

    # clear list contents
    notes.clear()
    rf_rents.clear()
    avg_rents.clear()
    unit_info.clear()

    # get listing title info
    title = soup.find("title").text.split(" | ")
    address = title[0]
    city = address.split(",")[1].replace(" ", "")
    mls = str(Decimal(sub(r'[^\d.]', '', title[1])))
    
    # locate city index to get average rent info
    try:
        if(city != ""):
            city_idx = int(cities.index(city))
    except:
        print("City: " + city + " is not in average region rent list")

    # get basic listing info
    stat_vals = soup.find_all(class_="statsValue")
    
    try:
        list_price = int(Decimal(sub(r'[^\d.]', '', stat_vals[0].text)))
    except:
        print("Login may be required to access this listing, skipping...\n")
        continue
    
    beds = stat_vals[1].text
    baths = stat_vals[2].text
    sqft = stat_vals[3].text

    # get RedFin estimate
    try:
        redfin_estimate_header = soup.find("div", class_="RedfinEstimateValueHeader").find_next("div", class_="value font-size-large")
        redfin_estimate = int(Decimal(sub(r'[^\d.]', '', redfin_estimate_header.text)))
    except:
        notes.append("Listing does not have a RedFin estimate, using list price")
        redfin_estimate = list_price  
        
    # get home facts, careful when using these for calculations as listing indicies can change
    home_facts = soup.find_all("span", class_="content text-right")   
    property_type = home_facts[0].text
    # print(property_type)
    try:
        num_of_units = int(Decimal(sub(r'[^\d.]', '', property_type)))
    except:
        notes.append("exception: " + property_type)
    
    # Greater than 4 units requires a commercial loan; skip these listings
    try:
        if(num_of_units > 6):
            print("Property has more than 6 units, skipping...\n")
            continue
    except:
        continue
        
    # status = home_facts[0].text
    # time_on_redfin = home_facts[1].text
    # year_built = home_facts[3].text
    # community = home_facts[4].text
    # lot_size = home_facts[5].text
    # commission = home_facts[11].text
      
    # get unit information
    for i in range(num_of_units):
        
        unit_beds = "0"
        unit_baths = "0"
        unit_rent = 0
        avg_rent = 0
        
        s = "Unit #" + str(i+1) + " Information"
        
        try:
            unit_info_header = soup.find(string=s).find_parents("ul")
            unit_info_list = str(unit_info_header).split("<li class=\"entryItem\"><span class=\"entryItemContent\">")
        except:
            notes.append("No individual unit info available")
            continue
            
        for item in unit_info_list:
            if("# of Bedrooms: " in item):
                unit_beds = int(Decimal(sub(r'[^\d.]', '', item)))
                
                # no data available for greater than 4 bedrooms
                if(unit_beds > 4):
                    notes.append("no MA rent data on greater than 4 bedrooms")
                else:
                    if(city != ""):
                        avg_rent = int(avg_mass_rent.iloc[city_idx][str(unit_beds) + "BR"])
                    else:
                        notes.append("city not found in average region rent list")
                
            if("# of Full Baths: " in item):               
                unit_baths = str(Decimal(sub(r'[^\d.]', '', item)))
                            
            if("Rent: " in item):
                unit_rent = int(Decimal(sub(r'[^\d.]', '', item)))
                # unit_rent_fmt = "${:,d}".format(unit_rent)
        
        rf_rents.append(unit_rent)
        avg_rents.append(avg_rent)
        unit_info.append({"beds": str(unit_beds), "baths": unit_baths})

    # get zestimate
    zillow_rent = getZestimate(driver, address) 
    # zillow_rent = getZestimate(driver, address)*(num_of_units-1) 

    # determine which rent estimate to use
    if(np.sum(rf_rents) == 0):
        if(zillow_rent == 0):
            mfh_rent = 0
            notes.append("Property does not have a zestimate, set rent to zero")  
        else:
            mfh_rent = zillow_rent  
            notes.append("Used zestimate in calculations")
    else:
        # sort the RedFin rents and only take the top (num_of_units - 1) rents
        # one unit's rent is reserved for the owner and is not accounted for
        mfh_rent = np.sum((np.sort(rf_rents)[::-1])[0:num_of_units-1])
    
    total_rent = mfh_rent + groveSt_rent
 
    # get property taxes 
    try:
        taxes_header = str(soup.find(string="Taxes: ").find_parents("span"))
        prop_taxes = int(Decimal(sub(r'[^\d.]', '', taxes_header)))/12
    except:
        notes.append("Property taxes not available for this listing, setting taxes to zero")
        prop_taxes = 0

    # multifamily mortgage calculations
    downpayment = list_price*downpayment_pct
    loan = list_price - downpayment
    closing_costs = loan*closing_costs_pct + downpayment
    insurance = (list_price*insurance_pct)/12
    pmi = (loan*pmi_pct)/12
    
    mortgage_pmt = -npf.pmt(rate/12, term*12, loan)
    mfh_piti = np.round(mortgage_pmt + prop_taxes + insurance + pmi, 2)
    
    # total monthly savings combined with 30 Grove St
    # most important metric b/c a positive value increases net worth
    total_piti = mfh_piti + groveSt_piti
    savings = np.round(total_rent - total_piti + groveSt_piti - expenses, 2)
    
    # cash flows
    cash_flow_combined = np.round(total_rent - total_piti - expenses, 2)
    cash_flow_mfh = np.round(mfh_rent - mfh_piti - expenses, 2)
    cash_on_cash_rtn = np.round(((cash_flow_mfh*12) / closing_costs)*100, 1)
    
    # cap rate of MFH only
    net_operating_income = (mfh_rent - expenses)*12
    cap_rate = np.round((net_operating_income / redfin_estimate)*100, 1)
    
    # determine if property has a good cap rate (8-12%)
    if(8 <= cap_rate <= 12):
        notes.append("Property has a good cap rate")
    
    print(unit_info)
    print("redfin rents: " + str(rf_rents))
    # print("monthly savings: $" + str(savings))
    print("MFH piti: $" + str(mfh_piti))
    # print("combined cash flow: $" + str(cash_flow_combined))
    print("MFH cash flow: $" + str(cash_flow_mfh))
    print("MFH CoC return: " + str(cash_on_cash_rtn) + "%")
    print("cap rate: " + str(cap_rate) + "%")
    
    if(mfh_rent == 0):
        isOfInterest = "See notes"
    if(savings > 0):
        print("Property DOES improve net worth, MLS: #" + mls)
        isOfInterest = True
    else:
        # print("Property DOES NOT improve net worth")
        isOfInterest = False

    data_out = {
        'MLS': [mls],
        'Address': [address],
        'Town/City':[city],
        'Property Type': [property_type],
        'List Price': [list_price],
        'RedFin Estimate': [redfin_estimate],
        'Beds': [beds],
        'Baths':[baths],
        'SQFT': [sqft],
        'Unit Info':[unit_info],
        'RedFin Rent Estimate per Unit':[rf_rents],
        'MA Average Rent per Unit': [avg_rents],
        'Zillow Rent Estimate':[zillow_rent],
        'Rent Estimate Used': [mfh_rent],
        'MFH Mortgage Payment': [mfh_piti],
        'Monthly Savings': [savings],
        'MFH Cash Flow': [cash_flow_mfh],
        'MFH Cash-on-Cash Return (%)': [cash_on_cash_rtn],
        'Combined Cash Flow': [cash_flow_combined]
        'Cap Rate (%)': [cap_rate],
        'Interested?': [isOfInterest],
        'URL': [url],
        'Notes': [notes]
    }   

    df_out = pd.DataFrame(data_out)
    df_out.to_csv(fn_out, mode='a', index=False, header=False)
    
    print("\n")

driver.quit()

print("SCRIPT DONE!")

1 of 350

https://www.redfin.com/NJ/Ridgewood/200-Claremont-Rd-07450/home/35841203
City: Ridgewood is not in average region rent list
2 of 350

https://www.redfin.com/NJ/Montvale/16-Stuyvesant-Rd-07645/home/35796005
City: Montvale is not in average region rent list
3 of 350

https://www.redfin.com/NJ/Ho-Ho-Kus/65-Jacquelin-Ave-07423/home/35767781
City: Ho-ho-kus is not in average region rent list
4 of 350

https://www.redfin.com/NJ/Bergenfield/134-Wilbur-Rd-07621/home/35682017
City: Bergenfield is not in average region rent list
5 of 350

https://www.redfin.com/NJ/Hackensack/136-Kent-St-07601/home/108170679
City: Hackensack is not in average region rent list
6 of 350

https://www.redfin.com/NJ/Westwood/67-Harding-Ave-07675/home/35884694
City: Westwood is not in average region rent list
7 of 350

https://www.redfin.com/NJ/River-Vale/712-Orangeburgh-Rd-07675/home/35847743
City: RiverVale is not in average region rent list
8 of 350

https://www.redfin.com/NJ/Ramsey/238-S-Central-Ave-07446

City: Closter is not in average region rent list
63 of 350

https://www.redfin.com/NJ/Alpine/19-Allison-Rd-07620/home/35675732
City: Alpine is not in average region rent list
64 of 350

https://www.redfin.com/NJ/Hillsdale/7-Royal-Park-Ter-07642/home/35764317
City: Hillsdale is not in average region rent list
65 of 350

https://www.redfin.com/NJ/Demarest/34-Woodland-Rd-07627/home/35698068
City: Demarest is not in average region rent list
66 of 350

https://www.redfin.com/NJ/Wallington/33-Pulaski-Ave-07057/home/35880302
City: Wallington is not in average region rent list
67 of 350

https://www.redfin.com/NJ/Fairview/18-Bergen-Blvd-07022/home/35731845


In [15]:
driver.get(df_in.iloc[0][0])